In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Iceberg Catalog Setup") \
    .getOrCreate()

print("Spark Running")
print(spark.sparkContext.getConf().getAll())
print("current catalog:", spark.catalog.currentCatalog())
print("Spark UI:", spark.sparkContext.uiWebUrl)

In [ ]:
import subprocess

# Define the SQL script
sql_script = """
CREATE EXTERNAL CATALOG 'iceberg'
PROPERTIES
(
  "type"="iceberg",
  "iceberg.catalog.type"="rest",
  "iceberg.catalog.uri"="http://iceberg-rest:8181",
  "iceberg.catalog.warehouse"="warehouse",
  "aws.s3.access_key"="admin",
  "aws.s3.secret_key"="password",
  "aws.s3.endpoint"="http://minio:9000",
  "aws.s3.enable_path_style_access"="true",
  "client.factory"="com.starrocks.connector.iceberg.IcebergAwsClientFactory"
);
"""

# Save the SQL script to a temporary file
with open("/tmp/script.sql", "w") as f:
    f.write(sql_script)

# Run the SQL script using docker compose and StarRocks
command = [
    "mysql", "-P", "9030", "-h", "starrocks-fe", "-u", "root", "--prompt=StarRocks > ", 
    "-e", "source /tmp/script.sql"
]

# Execute the command
result = subprocess.run(command, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
def clean_csv(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    with open(file_path, 'w') as file:
        for line in lines:
            file.write(line.rstrip().rstrip(',') + '\n')

In [ ]:
import os
from pathlib import Path

wbi_data_path = "/home/iceberg/data/world_bank_data"
csv_files = [file for file in os.listdir(wbi_data_path) if file.endswith(".csv")]

spark.sql("CREATE NAMESPACE IF NOT EXISTS raw")

for csv in csv_files:
  file_path = os.path.join(wbi_data_path, csv)
  clean_csv(file_path)
  file_name = Path(file_path).stem
  file_name = file_name.replace("-", "_")

  df = spark.read.option('header', 'true').csv(file_path)

  (df
  .writeTo(f"raw.world_development_indicators.{file_name}")
  .options(format="iceberg", mode="overwrite")
  .createOrReplace()
  )


In [ ]:
emissions_data_path = "/home/iceberg/data/emissions_data"

for year in [2017, 2018, 2019]:
  file_path = f"{emissions_data_path}/co2_emissions_passenger_cars_{year}.json"
  file_name = Path(file_path).stem

  df = spark.read.option("multiline","true").json(file_path)

  (df
  .writeTo(f"raw.co2_passenger_cars_emissions.{file_name}")
  .options(format="iceberg", mode="overwrite")
  .createOrReplace()
  )


In [ ]:
iceberg_data_df = spark.read.table("iceberg.raw.world_development_indicators.WDIData")
iceberg_co2_emissions_df = spark.read.table("iceberg.raw.co2_passenger_cars_emissions.co2_emissions_passenger_cars_2017")

iceberg_data_df.printSchema()
iceberg_co2_emissions_df.printSchema()

In [ ]:
print(f"Number of records  for CO2 emissions DF: {iceberg_co2_emissions_df.count()}")
print(f"Number of records  for World Development Indicators: {iceberg_data_df.count()}")

In [ ]:
display(iceberg_co2_emissions_df.describe())

In [ ]:
display(iceberg_data_df.describe())

In [ ]:
display(iceberg_data_df.head())

In [ ]:
display(iceberg_co2_emissions_df.head())

In [ ]:
spark.stop()